In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
import os


def set_environment_variables(env_path: str = '../.env',):
    with open(env_path, 'r') as f:
        key, val = f.readline().split()
        os.environ[key] = val

In [4]:
set_environment_variables()

In [5]:
import os, sys

print(os.environ.get("SHIT"))

None


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter 
from langchain.document_loaders import TextLoader

loader = TextLoader(['../preprocessedDocs/fixed_skillfactory.txt', "../burnout.txt"])
# loader = TextLoader('../burnout.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)
db

RuntimeError: Error loading ../preprocessedDocs/fixed_skillfactory.txt

In [ ]:
retriever = db.as_retriever()
retrieved_docs = retriever.invoke("Я хочу питсы")

retrieved_docs[0].page_content

'и хроническую усталость Если вы вовремя их обнаружите сможете переключиться взять отдых провести время с друзьями сменить локацию'

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [ ]:
llm = GigaChat(
    credentials=os.environ['GIGACHAT_API_KEY'],
    model="GigaChat",
    verify_ssl_certs=False,
    profanity_check=False,
)

prompt = ChatPromptTemplate.from_template(''' Ты - психолог помогающий бороться с выгоранием \
                                          Ответь на вопрос пользователя. \
Используй при этом информацию из контекста. Если в контексте нет \
информации для ответа, попроси пользователя рассказать про свои проблемы.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)
# prompt = ChatPromptTemplate.from_template('''Ну такое
# Контекст: {context}
# Вопрос: {input} 
# Ответ:''')

In [ ]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [ ]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
retrieval_chain.invoke({'input': "Можешь рассказать про выгорание в IT-сфере?"})

{'input': 'Можешь рассказать про выгорание в IT-сфере?',
 'context': [Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='совсем другие подходы и языки Так я стал комфортно чувствовать себя как профессионал что позитивно сказалось на основной работе'),
  Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='Профессиональное выгорание напрямую зависит от двух факторов  самого человека и проекта над которым он работает Думаю лучше разобрать второй фактор'),
  Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='3 Часто профессиональное выгорание следует за эмоциональным Если у человека много личных проблем и переживаний это сказывается на работе Сотрудник чувствует себя подавленным теряет силы и желание профессионально развиваться'),
  Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='и хроническую усталость Если вы вовремя их обнаружите с

In [ ]:
len(docs[0].page_content)

442

In [ ]:
len(documents)

1

In [ ]:
retriever.invoke("Можешь рассказать про выгорание в IT-сфере?")

[Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='совсем другие подходы и языки Так я стал комфортно чувствовать себя как профессионал что позитивно сказалось на основной работе'),
 Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='Профессиональное выгорание напрямую зависит от двух факторов  самого человека и проекта над которым он работает Думаю лучше разобрать второй фактор'),
 Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='3 Часто профессиональное выгорание следует за эмоциональным Если у человека много личных проблем и переживаний это сказывается на работе Сотрудник чувствует себя подавленным теряет силы и желание профессионально развиваться'),
 Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='и хроническую усталость Если вы вовремя их обнаружите сможете переключиться взять отдых провести время с друзьями сменить локац

bm25_chain

In [ ]:
import string


def tokenize(s):
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")


In [ ]:
from langchain.chains import RetrievalQA

embedding_retriever = retriever
bm25_retriever = BM25Retriever.from_documents(
    documents=docs,
    preprocess_func=tokenize,
    k=3,
)

bm25_chain = create_retrieval_chain(bm25_retriever, document_chain)

In [ ]:
bm25_chain.invoke({'input': "Можешь рассказать про выгорание в IT-сфере"})

{'input': 'Можешь рассказать про выгорание в IT-сфере',
 'context': [Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='Текущая ситуация в отрасли в основном влияет не на профессиональное выгорание а на сокращение количества IT-специалистов и рынка IT в целом в России В большей степени это можно назвать кризисными стресс-факторами Специалисты понимают что не смогут реализовать свои профессиональные ожидания и возможности'),
  Document(metadata={'source': '../preprocessedDocs/fixed_skillfactory.txt'}, page_content='Выгорание встречается и у аналитиков которые работают на проекте в большой команде и в основном занимаются подготовкой требований Зачастую их вклад в общей структуре бизнеса невелик а влияние на общий результат не так заметен Это видно в компаниях с высоким уровнем бюрократизма например в госсекторе или в финтехе Там специалисты часто не могут реализовать свои профессиональные ожидания показать творческие возможности и проявить инициатив